## Block & Layer

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0682, -0.3006,  0.0767, -0.2318,  0.4020,  0.1572, -0.1922, -0.1060,
          0.0586, -0.1504],
        [-0.1605, -0.1925,  0.1421, -0.1693,  0.4092,  0.1359, -0.1062, -0.3035,
          0.0589, -0.1546]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.2320, -0.0494,  0.0238,  0.0338,  0.0653, -0.3812, -0.2105, -0.0072,
         -0.0460,  0.1518],
        [-0.2444, -0.0225,  0.0161,  0.0215,  0.0928, -0.3726, -0.1644,  0.0646,
         -0.1366,  0.1970]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3080,  0.0109,  0.4660, -0.1799,  0.0493, -0.2557, -0.0657, -0.0498,
         -0.1048, -0.3709],
        [ 0.2133,  0.0389,  0.3233, -0.0918,  0.0370, -0.1637, -0.0829, -0.1349,
         -0.0165, -0.2271]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.0063, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    """All sorts of blocks combined"""
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0566, grad_fn=<SumBackward0>)

## Parameters Management

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.randn(size=(2, 4))
net(X)

tensor([[-0.1796],
        [ 0.0336]], grad_fn=<AddmmBackward0>)

In [10]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1565,  0.0970, -0.1195, -0.1693, -0.2805, -0.0107, -0.3383, -0.1761]])), ('bias', tensor([0.1439]))])


In [11]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(type(net[2].bias.data))

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1439], requires_grad=True)
tensor([0.1439])
<class 'torch.Tensor'>


In [12]:
net[2].weight.grad == None

True

In [13]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
net.state_dict()['2.bias'].data

tensor([0.1439])

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3708],
        [-0.3707]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data
# gnet[0][1][1].bias.data, it will break down. Because AttributeError: 'ReLU' object has no attribute 'bias'.

tensor([ 0.0653, -0.3765, -0.1017, -0.1801,  0.1406,  0.0337,  0.3081, -0.3765])

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0050,  0.0038,  0.0007,  0.0096]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_constant_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_constant)
print(net)
print(net[0].weight.data[0])
print(net[2].weight.data)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
tensor([ 0.4600,  0.1477, -0.3976, -0.5766])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.]])


$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) 
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.4580, -0.0000,  9.3121, -0.0000],
        [-9.0532, -0.0000, -0.0000,  9.6595]], grad_fn=<SliceBackward0>)

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, 10.3121,  1.0000])

## Parameters Binding

In [23]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
print(net[2].weight is net[4].weight)

True


## Deferred-Init

In [24]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
print(net)
print([net[i].state_dict() for i in range(len(net))])
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
[OrderedDict([('weight', <UninitializedParameter>), ('bias', <UninitializedParameter>)]), OrderedDict(), OrderedDict([('weight', tensor([[ 0.0020, -0.0244,  0.0011,  ...,  0.0154,  0.0414, -0.0144],
        [ 0.0578, -0.0320,  0.0252,  ...,  0.0385, -0.0236, -0.0395],
        [ 0.0179,  0.0576,  0.0083,  ..., -0.0279, -0.0167,  0.0407],
        ...,
        [ 0.0420,  0.0320, -0.0491,  ..., -0.0165,  0.0113, -0.0408],
        [-0.0620,  0.0054, -0.0286,  ..., -0.0023,  0.0037,  0.0546],
        [-0.0268,  0.0026,  0.0377,  ...,  0.0361, -0.0168, -0.0569]])), ('bias', tensor([ 0.0236, -0.0391, -0.0360,  0.0605,  0.0539, -0.0424, -0.0352, -0.0522,
         0.0333,  0.0466]))])]
Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


c:\ProgramData\anaconda3\envs\d2l\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Custom-Layer

In [25]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [26]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [27]:
net = nn.Sequential(nn.Linear(8, 128), layer)

In [28]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-5.1223e-09, grad_fn=<MeanBackward0>)

In [29]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [30]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.7038, -1.1793,  0.9012],
        [ 0.0861, -0.5357,  1.3077],
        [-0.5639,  0.6680,  0.7077],
        [ 0.5269, -2.2602,  1.1254],
        [ 0.5494, -0.5316, -0.6157]], requires_grad=True)

In [31]:
linear(torch.rand(2, 5))

tensor([[1.7894, 0.0000, 3.3908],
        [1.3991, 0.0000, 2.5496]])

In [32]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[3.0662],
        [3.7879]])

$y_k = \sum_{i, j} W_{ijk} x_i x_j$

In [33]:
class lowdimlayer(nn.Module):
    def __init__(self, in_unit) -> None:
        super().__init__()
        self.in_unit = in_unit
        self.weight = nn.Parameter(torch.randn(in_unit, in_unit, in_unit))

    def forward(self, X):
        assert X.shape[0] == self.in_unit
        y = torch.zeros_like(X)
        for k in range(0, X.shape[0]):
            for i in range(0, X.shape[0]):
                for j in range(0, X.shape[0]):
                    y[k] += self.weight[i][j][k] * X[i] * X[j]
        return y
        
# Test
net = lowdimlayer(3)
print(net.weight)
print(net(torch.randn(3, 2, 3)))

Parameter containing:
tensor([[[-0.5815, -0.5562,  1.0808],
         [ 2.3822,  1.2935, -0.5762],
         [-0.1566,  0.0896, -1.1056]],

        [[-0.3453, -1.8915, -2.0096],
         [-0.1673,  1.1895,  0.2131],
         [ 0.4965,  0.9301, -0.1089]],

        [[-0.4860, -0.8089, -0.9093],
         [-0.3853, -0.0677,  0.7947],
         [ 0.4478,  1.2075,  1.6336]]], requires_grad=True)
tensor([[[ 0.5648,  0.2963, -0.4813],
         [ 1.0415, -1.3843, -1.8713]],

        [[ 2.0227,  0.9303,  0.4260],
         [ 5.3321,  1.5236,  0.9184]],

        [[-0.2003,  1.2792,  0.8328],
         [ 3.4206,  1.7768,  2.5707]]], grad_fn=<CopySlices>)


In [34]:
class flourierlayer(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, X):
        fft = torch.fft.fft(X)
        return fft[:, :round(X.shape[1] / 2)]
    
        
# Test
net = flourierlayer()
print(net(torch.randn(3, 2)))

tensor([[-1.0621+0.j],
        [-1.7471+0.j],
        [ 1.2633+0.j]])


## Read & Write

In [35]:
x = torch.arange(4)
# torch.save(x, 'x-file')
# x1 = torch.load('x-file')

y = torch.zeros_like(x)
# torch.save([x, y], 'x-files')
# x1, y1 = torch.load('x-files')

mydict = {'x': x, 'y': y}
# torch.save(mydict, 'mydict')
# mydict2 = torch.load(mydict) 

In [36]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [37]:
# torch.save(net.state_dict(), 'mlp.params')

In [38]:
clone = MLP()
# clone.load_state_dict(torch.load('mlp.params'))
clone

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [39]:
Y_clone = clone(X)
Y_clone == Y

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]])

## GPU

In [40]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [41]:
torch.cuda.device_count()

1

In [42]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [43]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [44]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1579, 0.9118, 0.0451],
        [0.8437, 0.6062, 0.5344]])

In [45]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.1579, 0.9118, 0.0451],
        [0.8437, 0.6062, 0.5344]])
tensor([[0.1579, 0.9118, 0.0451],
        [0.8437, 0.6062, 0.5344]], device='cuda:0')


In [46]:
X + Z

tensor([[1.1579, 1.9118, 1.0451],
        [1.8437, 1.6062, 1.5344]], device='cuda:0')

In [47]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [48]:
net(X)

tensor([[-0.9725],
        [-0.9725]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [49]:
net[0].weight.data.device

device(type='cuda', index=0)

In [50]:
import time
start_cpu = time.time()
for i in range(1000):
    A = torch.randn(1000, 1000)
    B = torch.randn(1000, 1000)
    C = torch.matmul(A, B)
end_cpu = time.time()

time.sleep(5)

start_gpu = time.time()
for i in range(1000):
    A = torch.randn(1000, 1000, device=try_gpu())
    B = torch.randn(1000, 1000, device=try_gpu())
    C = torch.matmul(A, B)
end_gpu = time.time()

print(f'大计算量任务CPU运算时长: {round((end_cpu - start_cpu)*1000, 2)} ms')
print(f'大计算量任务GPU运算时长: {round((end_gpu - start_gpu)*1000, 2)} ms')

大计算量任务CPU运算时长: 36084.4 ms
大计算量任务GPU运算时长: 1231.88 ms


In [51]:
start_cpu = time.time()
A = torch.randn(5, 5)
B = torch.randn(5, 5)
C = torch.matmul(A, B)
end_cpu = time.time()

time.sleep(5)

start_gpu = time.time()
A = torch.randn(5, 5, device=try_gpu())
B = torch.randn(5, 5, device=try_gpu())
C = torch.matmul(A, B)
end_gpu = time.time()

print(f'小计算量任务CPU运算时长: {round((end_cpu - start_cpu)*1000, 2)} ms')
print(f'小计算量任务GPU运算时长: {round((end_gpu - start_gpu)*1000, 2)} ms')

小计算量任务CPU运算时长: 2.05 ms
小计算量任务GPU运算时长: 19.79 ms


In [52]:
A = torch.rand(100, 100, device=try_gpu())

start = time.time()
for i in range(1000):
    A = torch.mm(A, A)
    B = torch.norm(A)  # 逐个记录
end = time.time()
print(f'逐个记录耗费时间：{round((end - start) * 1000)} ms')

A = torch.rand(100, 100, device=try_gpu())
start = time.time()
for i in range(1000):
    A = torch.mm(A, A)
B = torch.norm(A)  # 最终记录
end = time.time()
print(f'最终记录耗费时间：{round((end - start) * 1000)} ms')

逐个记录耗费时间：226 ms
最终记录耗费时间：113 ms
